# ECH - Encuesta continua de hogares 2020

## Objetivos

En esta investigación buscamos determinar cuánto afecta el nivel educativo, la cantidad de hijos y otros indicadores, a la inserción en el mercado laboral de las mujeres uruguayas. Luego de un análisis principal de la base para conocer el contexto general de la investigación, intentaremos crear un algoritmo de clasificación para entender la influencia de distintas variables categóricas en un resultado determinado.

### Librerias a utilizar

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# sns.set(rc={'figure.figsize':(30,15)})

### Importar dataset y seleccionar atributos de análisis principales.

Para leer el archivo es necesario contar con la libreria _pyreadstat_

In [5]:
df = pd.read_spss('/Users/anicolon/Desktop/coder_proyecto/ECH_2019/ECH_2019.sav')

df = df.rename(columns={
    'numero': 'id_hogar',
    'e26': 'sexo',
    'e27': 'edad',
    'e29_6': 'ascendencia',
    'e30': 'parentesco',
    'pobpcoac': 'estado_laboral',
    'g126_1': 'sueldo',
    'f99': 'busca_otro_trabajo',
    'f72_2': 'actividad',
    'e49_CV': 'asistencia a enseñanza',
    'e49': 'asistencia alguna vez a enseñanza',
    'e193': 'edu preescolar',
    'e197': 'edu primaria',
    'e201': 'edu media',
    'e212': 'edu técnica',
    'e215': 'edu mag prof',
    'e218': 'edu univ o simil',
    'e221': 'edu terciario no uni',
    'e224': 'edu posgrado',
    'NOMBARRIO': 'barrio',
    'e36': 'estado_civil',
    'e185': 'hijos',
    'e48': 'leer_escribir',
    'e186_1': 'hijos en hogar',
    'e186_2': 'hijos en otro hogar',
    'e186_3': 'hijos en el extranjero',
    'region_4': 'region',
    'estred13': 'estrato',
    'd8_1': 'vivienda'
    })

In [6]:
df = df[[
    'id_hogar', 
    'sexo', 
    'edad', 
    'ascendencia', 
    'parentesco', 
    'estado_laboral', 
    'nper', 
    'sueldo', 
    'busca_otro_trabajo',
    'actividad',
    'asistencia alguna vez a enseñanza',
    'edu preescolar',
    'edu primaria',
    'edu media',
    'edu técnica',
    'edu univ o simil',
    'edu terciario no uni',
    'edu posgrado',
    'edu mag prof',
    'barrio',
    'estado_civil',
    'hijos',
    'hijos en hogar',
    'hijos en otro hogar',
    'hijos en el extranjero',
    'leer_escribir',
    'region',
    'vivienda',
    'estrato'
    ]]

In [7]:
df.to_csv('ECH_2019.csv', index=None)

In [ ]:
df['sexo'] = df['sexo'].astype('category')
df['barrio'] = df['barrio'].astype('category')

## Primera exposición a los datos

In [ ]:
df.sample(2)

In [ ]:
df.info()

In [ ]:
df.describe().transpose()

### Ascendencia

In [ ]:
asc_princ_porcentage = df['ascendencia'].value_counts() / len(df['ascendencia']) * 100

asc_princ_porcentage

## Estudio de la composición de los hogares uruguayos

### Cantidad de personas que conforman un grupo familiar

In [ ]:
composicion_familiar = pd.pivot_table(df, index='id_hogar', columns='parentesco',values=['nper'], aggfunc='count')

composicion_familiar['totales'] = composicion_familiar.apply(lambda x:x.sum(), axis=1)

composicion_familiar = composicion_familiar.sort_values(by='totales', ascending=False)

composicion_familiar['totales'].value_counts()

## Organización familiar

In [ ]:
df['parentesco'].value_counts()

In [ ]:
jefes_de_hogar_sexo = df[df['parentesco'].str.contains('Jefe')]['sexo'].value_counts()

sns.barplot(x=jefes_de_hogar_sexo.index, y=jefes_de_hogar_sexo)

In [ ]:
df['suma_hijos'] = df[['hijos en hogar','hijos en otro hogar','hijos en el extranjero',]].apply(lambda x:x.sum(), axis=1)

df['suma_hijos'].value_counts()

In [ ]:
df.to_csv('ECH_2019.csv')

## Situación laboral

In [ ]:
df['estado_laboral'].value_counts()

In [ ]:
sns.countplot(x=df['estado_laboral'])

In [ ]:
df['estado_laboral'].value_counts() / len(df['estado_laboral']) * 100

In [ ]:
df.groupby(by='estado_laboral').mean()

In [ ]:
# df = df[~df['ascendencia'].str.contains('Sin dato')]

sns.boxplot(data=df, x='ascendencia', y='sueldo', hue='sexo')

In [ ]:
sns.lineplot(data=df, x='edad', y='sueldo',hue='sexo')

## Comienzo de análisis más profundo. 

En este punto nos empezamos a centrar en las madres, jefas de hogares.

In [ ]:
df['parentesco_num'] = df['parentesco'].cat.codes

df[['parentesco_num', 'parentesco']].value_counts()

Considerando Hijo/a de ambos, Hijo/a solo del jefe/a, Nieto/a, Hijo/a solo del esposo/a compañero/a para ampliar un poco la muestra y tener en cuenta que monomarental puede incluir a los hijos del compañero porque la jefa de hogar estaría a cargo de ellos.

In [ ]:
mask = df["parentesco_num"].isin([3, 4, 5, 6, 7])
df = df[mask]

df["monomarental"] = np.NaN
df["monomarental"] = (
    df["monomarental"]
    .apply(lambda x: "monomarental")
    .where(df["parentesco_num"] == 6)
    .where(df["sexo"].str.contains("Mujer"))
)

### Empezamos a centrarnos en las hogares monomarentales

In [ ]:
# Eliminamos hogares con jefes de hogar masculinos
df_monomarental_y_familia = df.copy().drop(
    labels=list(
        df.loc[(df["sexo"].str.contains("Hombre")) & (df["parentesco_num"] == 6)].index
    )
)

df_monomarental_y_familia.sample(5)

## Investigamos el contexto de las jefas de hogar activas laboralmente

In [ ]:
df_monomarental = df_monomarental_y_familia.copy()[df_monomarental_y_familia['monomarental'].notna()]

df_monomarental.sample(5)

df_monomarental[[
    'id_hogar', 
    'sexo', 
    'edad', 
    'ascendencia', 
    'parentesco', 
    'estado_laboral', 
    'nper', 
    'sueldo', 
    'busca_otro_trabajo',
    'actividad',
    'asistencia alguna vez a enseñanza',
    'edu preescolar',
    'edu primaria',
    'edu media',
    'edu técnica',
    'edu univ o simil',
    'edu terciario no uni',
    'edu posgrado',
    'edu mag prof',
    'barrio',
    'estado_civil',
    'hijos',
    'hijos en hogar',
    'hijos en otro hogar',
    'hijos en el extranjero',
    'suma_hijos',
    'leer_escribir'
    ]].to_csv('monomarental.csv', index=None)

In [ ]:
df_monomarental['estado_laboral'].value_counts()

In [ ]:
df_monomarental.groupby(by='estado_laboral').mean()

In [ ]:
sns.displot(df_monomarental['sueldo'])

### En la gráfica anterior nos enfrentamos a la necesidad de eliminar outliers para realizar un análisis más ajustado a la realidad general de las madres

In [ ]:
df_monomarental_sinoutliers = df_monomarental[
    df_monomarental["estado_laboral"].str.contains("Ocupados")
].copy()

df_monomarental_sinoutliers = df_monomarental_sinoutliers[
    df_monomarental_sinoutliers["edad"] > 14
]

df_monomarental_sinoutliers = df_monomarental_sinoutliers[
    df_monomarental_sinoutliers["sueldo"] > 0
]

In [ ]:
quantil_99 = df_monomarental_sinoutliers["sueldo"].quantile(0.99)
print(quantil_99)

df_monomarental_sinoutliers = df_monomarental_sinoutliers[
    df_monomarental_sinoutliers["sueldo"] < quantil_99
]

quartil_1 = df_monomarental_sinoutliers["sueldo"].quantile(0.25)
quartil_3 = df_monomarental_sinoutliers["sueldo"].quantile(0.75)

iqr = quartil_3 - quartil_1

df_monomarental_sinoutliers = df_monomarental_sinoutliers[
    df_monomarental_sinoutliers["sueldo"] < quartil_3 + 1.5 * iqr
]

df_monomarental_sinoutliers = df_monomarental_sinoutliers[
    df_monomarental_sinoutliers["sueldo"] > quartil_1 - 1.5 * iqr
]

In [ ]:
sns.displot(df_monomarental_sinoutliers['sueldo'])

### Sueldo por cantidad de hijos

In [ ]:
sns.boxplot(data=df_monomarental_sinoutliers, x='suma_hijos', y='sueldo')

plt.show()

In [ ]:
df_monomarental_sinoutliers[['edad', 'sueldo', 'suma_hijos']].describe().transpose()

### Sueldo por ascendencia

In [ ]:
sns.boxplot(data=df_monomarental_sinoutliers, x='ascendencia', y='sueldo')

In [ ]:
df_monomarental_sinoutliers.sample(5)